In [2]:

import pandas as pd
import numpy as np
import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from unidecode import unidecode
## ler arquivo json em pandas
df = pd.read_json('/home/mfelipemota/projects/pantanaldev/data/label-studio/data/export/project-1-at-2023-04-30-05-08-956f4e3c.json')

df = df.drop(['id'], axis=1)
## expandir coluna annotations
df = pd.concat([df.drop(['annotations'], axis=1), df['annotations'].apply(pd.Series)], axis=1)
## expandir coluna 0 e renomear para annotations
df = pd.concat([df.drop([0], axis=1), df[0].apply(pd.Series)], axis=1)
## expandir coluna result e renomear para result
df = pd.concat([df.drop(['result'], axis=1), df['result'].apply(pd.Series)], axis=1)
df = df.drop(['id'], axis=1)

## expandir coluna 0 e renomear para result
df = pd.concat([df.drop([0], axis=1), df[0].apply(pd.Series)], axis=1)
## expandir coluna value e renomear para value
df = pd.concat([df.drop(['value'], axis=1), df['value'].apply(pd.Series)], axis=1)
## dropar choices nulos
df = df.dropna(subset=['choices'])
## obter choices 
df['choices'] = df['choices'].apply(lambda x: x[0])

## expandir coluna data
df = pd.concat([df.drop(['data'], axis=1), df['data'].apply(pd.Series)], axis=1)

df_noticia_original = df.copy()


padrao_data_cepea = r"Cepea, \d{2}/\d{2}/\d{4} - "
df['noticia'] = df['noticia'].apply(lambda x: re.sub(padrao_data_cepea, '', x))

## remover a palavra 'cepea' das noticias
padrao_cepea = r"Cepea"
df['noticia'] = df['noticia'].apply(lambda x: re.sub(padrao_cepea, '', x, flags=re.IGNORECASE))

## remover numeros das noticias
padrao_numeros = r'[0-9]+'
df['noticia'] = df['noticia'].apply(lambda x: re.sub(padrao_numeros, '', x))

## noticia que contem a palavra 'soja'
df = df[df['titulo'].str.contains('soja', flags=re.IGNORECASE)]

## remover noticias com choice 'desclassificar'
df = df[df['choices'] != 'Desclassificar']
df.count()

file_upload                 236
drafts                      236
predictions                 236
meta                        236
created_at                  236
updated_at                  236
inner_id                    236
total_annotations           236
cancelled_annotations       236
total_predictions           236
comment_count               236
unresolved_comment_count    236
last_comment_updated_at       0
project                     236
updated_by                  236
comment_authors             236
completed_by                236
was_cancelled               236
ground_truth                236
created_at                  236
updated_at                  236
lead_time                   236
prediction                  236
result_count                236
unique_id                   236
last_action                   0
task                        236
project                     236
updated_by                  236
parent_prediction             0
parent_annotation             0
last_cre

In [3]:
# Selecionar apenas as colunas necessárias
columns_to_select = ['id', 'data', 'noticia', 'titulo', 'choices', 'unique_id']

df = df[columns_to_select]
df.dropna(subset=['noticia'])

# Pré-processamento dos dados
stop_words = set(stopwords.words('portuguese'))

def preprocess_text(text):
    # remover acentuação
    text = unidecode(text)
    # Remover pontuações
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenização
    words = word_tokenize(text.lower())
    # Remover stopwords
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

df['noticia'] = df['noticia'].apply(preprocess_text)

df_treino = df[:210]
df_validacao = df[210:]

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import AutoModel, AutoTokenizer

# Criar modelo de classificação de sentimento
vocab_size = 1000
embedding_dim = 100
max_length = 125
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(50, activation='relu'),
    keras.layers.Dense(25, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compilar o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Pré-processar os dados
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(df['noticia'])
sequences = tokenizer.texts_to_sequences(df['noticia'])
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')
labels = df['choices'].map({'Positiva': 1, 'Negativa': 0, 'Neutra': 0.5})

# Treinar o modelo
history = model.fit(padded_sequences, labels, epochs=100, validation_split=0.3)

# Avaliar o modelo
test_sequences = tokenizer.texts_to_sequences(df_validacao['noticia'])
padded_test_sequences = pad_sequences(test_sequences, maxlen=max_length, padding='post', truncating='post')
test_labels = df_validacao['choices'].map({'Positiva': 1, 'Negativa': 0, 'Neutra': 0.5})
test_loss, test_accuracy = model.evaluate(padded_test_sequences, test_labels)
print('Test accuracy:', test_accuracy)

# Salvar o modelo
model.save('modelo.h5')

2023-05-06 21:29:40.760916: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-06 21:29:40.804449: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-06 21:29:41.797710: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Epoch 1/100
6/6 [==============================] - 1s 48ms/step - loss: 0.6924 - accuracy: 0.4606 - val_loss: 0.6922 - val_accuracy: 0.4366
Epoch 2/100
6/6 [==============================] - 0s 13ms/step - loss: 0.6901 - accuracy: 0.4667 - val_loss: 0.6913 - val_accuracy: 0.4366
Epoch 3/100
6/6 [==============================] - 0s 12ms/step - loss: 0.6867 - accuracy: 0.4667 - val_loss: 0.6904 - val_accuracy: 0.4366
Epoch 4/100
6/6 [==============================] - 0s 13ms/step - loss: 0.6829 - accuracy: 0.4667 - val_loss: 0.6894 - val_accuracy: 0.4366
Epoch 5/100
6/6 [==============================] - 0s 19ms/step - loss: 0.6784 - accuracy: 0.4667 - val_loss: 0.6879 - val_accuracy: 0.4366
Epoch 6/100
6/6 [==============================] - 0s 18ms/step - loss: 0.6725 - accuracy: 0.4667 - val_loss: 0.6858 - val_accuracy: 0.4366
Epoch 7/100
6/6 [==============================] - 0s 22ms/step - loss: 0.6651 - accuracy: 0.4667 - val_loss: 0.6822 - val_accuracy: 0.4366
Epoch 8/100
6/6 [===

In [4]:
## criar uma tabela de predições
df_predicoes = df_validacao.copy()
df_predicoes['predicao'] = model.predict(padded_test_sequences)
df_predicoes['predicao'] = df_predicoes['predicao'].apply(lambda x: 'Positiva' if x > 0.5 else 'Negativa' if x < 0.5 else 'Neutra')
df_predicoes

##  quantas noticias foram classificadas como positivas, negativas e neutras
df_predicoes['predicao'].value_counts()

## quantas noticias foram classificadas erradas
df_predicoes

1/1 [==============================] - 0s 109ms/step


,id,data,noticia,titulo,choices,unique_id,predicao
257,pQtBUDUD6I,4/10/2010,chuvas registradas boa parte pais ultimos dias...,SOJA/CEPEA: Clima seco em MT ainda preocupa,Negativa,e183012a-c39b-4609-8901-b168969ee53a,Negativa
263,wg007o9U_l,,precos soja derivados cairam mercado brasileir...,SOJA/CEPEA: Preços caem mesmo com atraso da co...,Negativa,a240499e-c9c9-47ea-9978-12cdb0ef9a2d,Negativa
265,cLvxr6IWgx,19/03/2007,indicador soja esalq mercado lotes parana caiu...,SOJA: Indicador cai 2% em sete dias,Negativa,d09539ea-f73c-4a11-a2c2-92c892e45763,Negativa
266,uUNzG2Aq7t,08/01/2007,chuvas principais regioes produtoras soja bras...,SOJA: Clima beneficia lavouras,Positiva,d6e77e6e-427a-4d61-8106-9a0694381a1a,Negativa
267,mK2V-OT8b8,22/01/2007,precos internos soja subiram ultimos dias inic...,SOJA: Prêmios limitam alta de preços,Positiva,0b135750-bc28-4a0d-a9de-aa82737b917f,Positiva
269,-hivWwwlgA,04/12/2006,daqui pra frente produtores devem focados clim...,SOJA: Fitossanidade preocupa setor,Negativa,7b63f020-2227-4618-988a-bacc86185f2f,Negativa
273,MnN7rKLL5W,02/01/2019,temporada soja registrando producao superior a...,SOJA/CEPEA: Preços se sustentam ao longo de 2018,Neutra,5e83462c-babc-4838-9624-dcd7dc44c54b,Negativa
276,rvYNosmiR-,26/12/2018,ainda ritmo lento parte produtores regiao nort...,SOJA/CEPEA: Colheita tem início em MT e deve s...,Neutra,2eb88c4f-2710-4b2b-82f4-cd8f3f9186ce,Positiva
279,ksRGb9gGig,30/07/2018,precos soja estao queda mercado brasileiro aco...,"SOJA/CEPEA: Com baixa liquidez, preços caem no...",Negativa,b91d6110-b000-46f6-afea-790d74223b1e,Negativa
286,rASALqlqS3,06/03/2019,recentes chuvas atrapalhado trabalhos campo ge...,"SOJA/CEPEA: Caso persista, clima chuvoso pode ...",Negativa,0f0f27f4-2a4c-4588-b6c3-d127fe776aa0,Negativa


In [5]:
## prever uma noticia
df_validacao = df_validacao.reset_index(drop=True)
noticia = df_validacao['noticia'][0]
noticia = preprocess_text(noticia)
noticia = [noticia]
noticia = tokenizer.texts_to_sequences(noticia)
noticia = pad_sequences(noticia, maxlen=max_length, padding='post', truncating='post')
val = model.predict(noticia)[0][0]


if val > 0.5:
    print('Positiva')
elif val < 0.5:
    print('Negativa')
else:
    print('Neutra')



1/1 [==============================] - 0s 29ms/step
Negativa


In [7]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Carregar o modelo pré-treinado BERTimbau
model = BertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased', num_labels=3)
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Definir otimizador e função de perda
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()

train_sentences = df_treino['noticia'].tolist()
train_labels = df_treino['choices'].map({'Positiva': 1, 'Negativa': 0, 'Neutra': 2}).tolist()
test_sentences = df_validacao['noticia'].tolist()
test_labels = df_validacao['choices'].map({'Positiva': 1, 'Negativa': 0, 'Neutra': 2}).tolist()

# Tokenizar as sentenças
train_encodings = tokenizer(train_sentences, truncation=True, padding=True, return_tensors='pt')
test_encodings = tokenizer(test_sentences, truncation=True, padding=True, return_tensors='pt')

# Criar um dataset do pytorch
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = MyDataset(train_encodings, train_labels)
test_dataset = MyDataset(test_encodings, test_labels)

# Treinar o modelo
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
for epoch in range(2):
    for i, batch in enumerate(train_loader):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optimizer.step()

# Avaliar o modelo nos dados de teste
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs[0], axis=1)
        total += labels.size(0)
        correct += (predictions == labels).sum().item()
    accuracy = correct / total
    print('Test accuracy:', accuracy)

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the

Test accuracy: 0.5
